# Building the Question Answering System

In [1]:
%load_ext autotime
import warnings; warnings.filterwarnings('ignore')

In [2]:
# %%capture
# from tqdm import tqdm_notebook as tqdm
# tqdm().pandas()

time: 596 ms


In [3]:
import numpy as np
from tqdm import tqdm
from collections import Counter
from preprocessing import *
from dmn import *
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


time: 2.61 s


In [4]:
def get_max_length(tokenized_contexts):
    max_context_len = np.max([len(context) for context in tokenized_contexts])
    return max_context_len    

time: 3.51 ms


In [5]:
prep = PreProcess(trainfile = "./Data/training_data.json", \
                  testfile  = "./Data/validation_data.json", \
                  glovefile = "./Data/glove.6B.50d.txt")

contexts, questions, answers, answer_start, answer_end = prep.get_data(train=True)

time: 5.27 s


In [6]:
tokenized_contexts, tokenized_questions, tokenized_answers, \
    sent_end_indices = prep.process(contexts, questions, answers) 

time: 4min 5s


In [ ]:
# define maodel and parameters
num_of_epochs = 25
batch_size = 50
num_of_samples = len(contexts)
num_of_batches = int(num_of_samples/batch_size)
max_context_len = get_max_length(tokenized_contexts)
max_question_len = get_max_length(tokenized_questions)
max_answer_len = get_max_length(tokenized_answers)
max_sent_num = get_max_length(sent_end_indices)
output_dim = get_max_length(contexts)

DMN_model = DynamicNetworkMemory(max_context_len, max_question_len, \
                                 max_sent_num, output_dim, summary=True)

time: 184 ms


In [ ]:
history_objects = []
batch_iteration = 0
for epoch in range(num_of_epochs): #change to num of epochs after
    for batch_idx in tqdm(range(num_of_batches)):
        print("\n Epoch %d/%d   Batch %d/%d" % (epoch+1, num_of_epochs, batch_iteration+1, num_of_batches))
        start, end = (batch_idx*batch_size, (batch_idx+1)*batch_size)    
        y_answer_start, y_answer_end = DMN_model.get_answer_span(answer_start[start:end], \
                                                                 answer_end[start:end])
        padded_contexts = prep.get_padded_inputs(tokenized_contexts[start:end], string_type="context")
        padded_questions = prep.get_padded_inputs(tokenized_questions[start:end], string_type="question")
        padded_sent_end_indices = pad_sequences(sent_end_indices[start:end], max_sent_num,
                                                                    padding="post", dtype="int32")
        
        history = DMN_model.fit(x = [padded_contexts, padded_sent_end_indices, padded_questions],
                                y = [y_answer_start, y_answer_end], verbose=0,
                                batch_size=batch_size, validation_split=0.1)
        
        batch_iteration += 1
    
    history_objects.append(history)
    batch_iteration = 0 # Reset batch iteration counter for each new epoch

  0%|          | 0/17 [00:00<?, ?it/s]


 Epoch 1/25   Batch 1/17
